Team2_Pytorch：109062676劉廷哲 110065508李丞恩 110062592姜宏昀 110062539古之恒

## Pick 5 descriptions from testing data and generate 5 images with different noise  z  respectively.

1. this white and purple flower has fragile petals and soft stamens 
2. this flower has four large wide pink petals with white centers and vein like markings 
3. a flower with broad white and pink ribbed petals and yellow stamen 
4. one prominet pistil with alarger stigam and many stamens with anthers 
5. leaves are green in color petals are light pink in color 

In [4]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://raw.githubusercontent.com/BennyNTHU/DL2021/main/Cup03/report.png?token=AH7NXYJB6JGK5Y36775FRLDB2WF24")

## Models you tried during competition. Briefly describe the main idea of the model and the reason you chose that model.

### Text encoder
我們使用的是tensorflow 提供的 roberta-large 來當作pretrained model，並預先算出所有caption的embedding，我們沒有將text encoder一起加入訓練，因為那樣太佔memory了，雖然可能一起訓練會好很多，但資源無法讓我們一起訓練

### Generator & Discriminator

- 1. 最開始試的是Lab14-2的架構，但馬上就發現效果有限
- 2. 在了解多一點後，其實generator就是一連串2d的deconvolution的過程，由已知的text_embedding和noise結合的訊息，經過NN，反向生成fake image。而discriminator正相反，是一連串2d的convolution，經過NN，最後和已知的text_embedding結合，再透過dense layer output出一個辨識score。$^{[1]}$
-> generator架構: concat_text -> (batchnormalization -> (leaky) relu -> conv2dtranspose)* -> activation -> fake image
-> discriminator架構: fake image -> (convolution2d(stride=2) -> batchnormalization -> "leaky" relu)* -> concat_text -> () -> score
- 3. lab14-2也是類似上面這個架構，只是network比較淺、kernel數量比較少。於是我們把層數增加，每層的kernel也增加，在參考這邊的架構$^{[2]}$和實驗後，建構了五層上述的generator&discriminator的架構，得到了現在的結果。

### Loss Function

- 使用的是和lab14-2作業相同的WGAN with gradient penalty loss

### Training

- 和Lab一樣，Discriminator和Generator的比例是5:1

## List the experiment you did. For example, data augmentation, hyper-parameters tuning, architecture tuning, optimizer tuning, and so on.

* data augmentation$^{[3]}$
    1. 因為我們認為花的照片水平翻轉應該也不影響觀看，以及不會改變其特性，所以我們有做水平翻轉
    2. 隨機調整亮度!
    3. 原先範例的code是從很多captions中random挑一個對應一張image，而我們選擇是全部的caption都會對應一張image，所以對相同augmented 的image有很多不同的captions
    4. 最後dataset大小大約會是原先的30倍
    
    
* Optimizer:
    1. generator和discriminator都是Adam()，beta=0.5，而learning rate也是1e-4
    
* GAN:
    1. 除此之外，在GAN的架構上，我們參考了這篇$^{[4]}$。參考丞恩在工研院實習的經驗，他曾經發現ResNet50在flower102 dataset有很好的表現，因此把discriminator的架構改成ResNet50、使用ResNet101作為generator。


# reference
[1]. https://ithelp.ithome.com.tw/articles/10227196

[2]. https://github.com/aditya30394/Reverse-Image-Captioning/blob/a6e427624a64f28d08e5629f48850ff001e48d02/nets/discriminator.py#L7

[3] W. Jiang, N. Ying. Improve Object Detection by Data Enhancement based on Generative Adversarial Nets, https://arxiv.org/ftp/arxiv/papers/1903/1903.01716.pdf

[4] N.T. Tran et al. On Data Augmentation for GAN Training, https://arxiv.org/pdf/2006.05338.pdf
